### GASS evolution

nb to test the evolution functions 

In [1]:
using PyCall
using DataFrames
using Printf

#rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"
rootdir = "/home/stephane/alma/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS
using Printf

import PyPlot
import Random

@pyimport astropy.coordinates as coord
@pyimport astropy.modeling.models as am
@pyimport astropy.modeling.fitting as mf

@pyimport numpy as np


## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:400
└ @ PyCall /home/stephane/.julia/packages/PyCall/RQjD7/src/PyCall.jl:400


In [2]:
mutable struct _population
    age::Int32                 ## age
    subarr::Array{Array{Int,1},2}     ## {Population,Subindices}
    fitness::Array{Float64,2}  ## fitness of each subarray
    score::Array{Float64,1}    ## global score of each set
    param::Array{Dict{String,Float64},2}    ## beam and mrs for each subarray    
end

In [3]:
## fitness function for a subarray
## cfg: GASS parameters
## subarrid: subarray id (int)
## subind: subarray indices in the main Arr.
##
function _fitness_subarray(cfg, subarrid, subind)
    subarr = cfg.arr[subind,:]
    
    bl= calc_baselines(subarr)
    uv= calc_uv(bl, cfg.obs.Source_Hour_Angle ,  cfg.obs.Source_Declination)
    h , dr=  calc_dirtybeam(uv , 255, 127, robust=0.5)
    b= fit_beam(h , dr)
    mrs= calc_mrs(uv)
    
    #@printf("## subarray fitness \n")
    #@printf("## beam:")
    #println(b)
    #@printf("## MRS: %3.3f \n", mrs)
    
    res= 0
    res += cfg.wei.Weight_Spatial_Resolution[subarrid]*abs(b.ar-cfg.sub.Spatial_Resolution[subarrid])
    res += cfg.wei.Weight_Elongation[subarrid]*abs(b.e-cfg.sub.Elongation[subarrid])*sign(b.e-
        cfg.sub.Elongation[subarrid])
    res += cfg.wei.Weight_Sidelobe_Levels[subarrid]*abs(b.sidelobe-
        cfg.sub.Sidelobe_Level[subarrid])*sign(b.sidelobe-cfg.sub.Sidelobe_Level[subarrid])
    res += cfg.wei.Weight_Maximum_Recoverable_Scale[subarrid]*abs(mrs-
        cfg.sub.Maximum_Recoverable_Scale[subarrid])*sign(cfg.sub.Maximum_Recoverable_Scale[subarrid]-mrs)
    
    return(res , [b , mrs])
end

_fitness_subarray (generic function with 1 method)

In [4]:
## Creation pf a population
## 

function _create_population(cfg)
    
    pop= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    paramsub= Array{Dict{String,Float64},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    
    subind= collect(1:cfg.obs.Antenna_Number)
    
    for i in 1:cfg.ga.Population_Size
        Random.shuffle!(subind)
        for j in 1:cfg.obs.Subarray_Number
            pop[i,j]= subind[cfg.sub.Subrange[j]]
            fitness[i,j] , res= _fitness_subarray(cfg, j, pop[i,j])
            println(i," ",j," ",fitness[i,j])
            println(res)
            paramsub[i,j]= Dict("ar"=>res[1].ar,"e"=>res[1].e, "sidelobe"=>res[1].sidelobe, "mrs"=>res[2])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end
    
    _pop= _population(0, pop, fitness, score , paramsub)
    
    return(_pop)
end

_create_population (generic function with 1 method)

In [5]:
## get best parents
##

function _get_elitism(cfg, pop::_population)
    isort= reverse(sortperm(pop.score))
    
    elit= []
    for i in 1:cfg.ga.Number_Elitism
        push!(elit, [pop.subarr[isort[i],:], pop.fitness[isort[i],:], pop.score[isort[i]]])
    end
    return(elit)
end

## using tournament selection
function _get_parents(cfg, pop::_population)
    tour= cfg.ga.Tournament_Size
    
    popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
    isort= reverse(sortperm(pop.score[popran[1:tour]]))
    iparent1= popran[isort[1]]
  
    iparent2= iparent1
    while iparent1==iparent2
        popran= Random.shuffle!(collect(1:cfg.ga.Population_Size))
        isort= reverse(sortperm(pop.score[popran[1:tour]]))
        iparent2= popran[isort[1]] 
    end 
    return(iparent1 , iparent2)
end

_get_parents (generic function with 1 method)

In [6]:
function _flatten_subarray(cfg, subarr)
    subflat= zeros(Int, cfg.obs.Antenna_Number)
    
    counter= 1
    for i in 1:cfg.obs.Subarray_Number
        for j in 1:cfg.sub.Pads_Per_Subarray[i]
            subflat[counter]= subarr[i][j]
            counter += 1
        end
    end
    return(subflat)
end

## reform the subarray array..
function _wrap_subarray(cfg, sub)
    subarray= Array{Array{Int,1}}(undef,0)

    for i in 1:cfg.obs.Subarray_Number
        push!(subarray, sub[cfg.sub.Subrange[i]])
    end
    
    return(subarray)
end

## crossover using the two parents
##
function _get_crossover1(cfg, pop::_population , parents)
    nmax= cfg.obs.Antenna_Number
    p1= _flatten_subarray(cfg, pop.subarr[parents[1],:])
    p2= _flatten_subarray(cfg, pop.subarr[parents[2],:])
    child= zeros(Int, nmax)

    pivot= rand(1:nmax,2)
    pivot1= minimum(pivot) ; pivot2= maximum(pivot)
    child[pivot1:pivot2] .= p1[pivot1:pivot2]

    irange1= 1:pivot1-1 ; irange2= pivot2+1:nmax
    
    cnterp2= 1
    for i in irange1
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end
    for i in irange2
        while (p2[cnterp2] in child)
            cnterp2 += 1
        end
        child[i]= p2[cnterp2]
    end  
    
    # println(child)
    c= _wrap_subarray(cfg, child)
    # println(c)
    return(child)
end

_get_crossover1 (generic function with 1 method)

In [7]:
## mutation
## loop over all alleles and swap two if p is true
## 

function _get_mutation(cfg, child)
    nmax= cfg.obs.Antenna_Number
    si= copy(child)
    
    pmutation= 1-cfg.ga.Mutation_Rate
    rng = Random.MersenneTwister()
    rd= Random.rand(rng, nmax)
    # println(rd)
    for i in 1:nmax
        if rd[i] > pmutation
            iswap= Random.rand(rng, big.(1:nmax))
            value= si[iswap]
            si[iswap]= si[i]
            si[i]= value
        end
    end
    return(si)
end

_get_mutation (generic function with 1 method)

In [8]:
## Evolve one population
##

function _get_evolution(cfg, pi::_population)
    npop= cfg.ga.Population_Size
    age= pi.age + 1
    nelit= cfg.ga.Number_Elitism
    
    ## Elitism
    pelit=  _get_elitism(cfg, pi)
    
    ## parent selection, crossover and mutation
    ncross= npop-nelit
    
    crosspop= []
    for i in 1:ncross
        parent= _get_parents(cfg, pi)
        child=  _get_crossover1(cfg, pi , parent)
        mutated= _get_mutation(cfg, child)
        push!(crosspop, mutated)
        println(parent)
        println(child)
    end

    
    ## create an evolved population
    pinew= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    paramsub= Array{Dict{String,Float64},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    
    for i in 1:nelit
        for j in 1:cfg.obs.Subarray_Number
            pinew[i,j]= pelit[i][1][j]
            fitness[i,j], res= _fitness_subarray(cfg, j, pinew[i,j])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end    
    
    println("fitness of crossover..")
    for i in nelit+1:npop
        println(crosspop[i-nelit])
        pwrap= _wrap_subarray(cfg, crosspop[i-nelit])
        println(pwrap)
        for j in 1:cfg.obs.Subarray_Number
            pinew[i,j]= pwrap[j]
            fitness[i,j] , res= _fitness_subarray(cfg, j, pinew[i,j])
            paramsub[i,j]= Dict("ar"=>res[1].ar,"e"=>res[1].e, "sidelobe"=>res[1].sidelobe, "mrs"=>res[2])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
    end
    
    _pop= _population(age, pinew, fitness, score, paramsub)
    
    return(_pop)
end

_get_evolution (generic function with 1 method)

In [ ]:
macro main(inpfile)
    cfg = read_cfg(inpfile , verbose=true)
    
    println("##")
    println("## Creating the population...")
    p0= _create_population(cfg)
    
    println("Elitism..")
    pelit=  _get_elitism(cfg, p0)
    
    println("Crossover...")
    parent= _get_parents(cfg, p0)
    child=  _get_crossover1(cfg, p0 , parent)
    println(child)
    
    println("Mutatis mutandis...")
    mutated= _get_mutation(cfg, child)
    
    ## evolution
    println("Evolution...")
    
    species= []
    for i in 1:cfg.ga.Number_Iterations
        p1= _get_evolution(cfg, p0)
        push!(species,p1)
        p0=p1
        if (i % 10) == 0
            printf("## Iteration: %d",i)
        end
    end
    
    
    ## PLOT ####
    #PyPlot.imshow(h)
    #PyPlot.colorbar()
    #PyPlot.show()
    
end

@main("../master/data/GA_Inputs_O-3.txt.julia")

## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", " Sub_2 ", " Sub_3 ", " Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 10 
### Population size: 20 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##
##
## Creating the population...
1 1 -1.315501680011497
Any[synthbeam(2.64259, 2.75099, 2.69624, 1.04102, 7.08903), 22.5883]
1 2 0.6987194344180123
Any[synthbeam(3.50781, 2.

Crossover...
[37, 46, 5, 32, 45, 4, 43, 6, 27, 10, 17, 49, 16, 1, 39, 20, 44, 18, 48, 31, 19, 25, 8, 35, 28, 2, 9, 22, 12, 34, 7, 29, 38, 11, 41, 15, 23, 21, 47, 42, 13, 50, 33, 24, 14, 26, 36, 3, 40, 30]
Mutatis mutandis...
Evolution...
(15, 11)
[32, 35, 20, 15, 50, 14, 41, 43, 6, 27, 10, 17, 49, 16, 1, 39, 25, 23, 37, 7, 5, 34, 2, 18, 33, 19, 9, 47, 8, 13, 42, 48, 46, 28, 31, 40, 45, 21, 11, 26, 36, 24, 12, 38, 30, 22, 4, 44, 3, 29]
(8, 11)
[32, 35, 39, 15, 41, 25, 43, 1, 23, 5, 49, 6, 44, 20, 45, 9, 4, 3, 21, 18, 2, 48, 14, 10, 50, 13, 36, 29, 7, 37, 34, 27, 33, 19, 47, 8, 17, 16, 42, 46, 28, 31, 40, 11, 26, 24, 12, 38, 30, 22]
(7, 5)
[38, 31, 30, 26, 24, 29, 33, 20, 16, 37, 1, 9, 42, 5, 7, 46, 50, 19, 47, 21, 25, 3, 10, 13, 14, 40, 23, 4, 6, 15, 39, 18, 48, 41, 36, 34, 44, 8, 12, 43, 35, 32, 49, 22, 17, 45, 27, 28, 2, 11]
(7, 8)
[34, 22, 15, 41, 24, 42, 47, 48, 7, 2, 45, 32, 46, 9, 19, 3, 33, 26, 14, 23, 1, 25, 28, 31, 20, 18, 11, 40, 21, 27, 36, 49, 6, 39, 37, 30, 44, 8, 12, 43, 3

14 4 6.4024638271894405
[34, 39, 28, 43, 30, 11, 33, 40, 23, 49, 44, 16, 3, 21, 18, 2, 48, 25, 35, 32, 8, 5, 9, 45, 10, 20, 6, 14, 50, 13, 36, 29, 7, 46, 1, 19, 4, 38, 22, 17, 47, 41, 42, 37, 12, 24, 27, 31, 26, 15]
Array{Int64,1}[[34, 39, 28, 43, 30, 11, 33, 40, 23, 49, 44, 16, 3, 21, 18, 2, 48, 25, 35, 32, 8, 5, 9, 45, 10, 20, 6, 14, 50, 13], [36, 29, 7, 46, 1, 19, 4, 38, 22, 17], [47, 41, 42, 37, 12, 24, 27], [31, 26, 15]]
15 1 -3.643982403672095
15 2 0.14414576505701282
15 3 -10.323632806703378
15 4 -12.607774013440956
[5, 47, 2, 48, 29, 12, 19, 21, 35, 16, 28, 24, 32, 44, 7, 50, 33, 37, 1, 9, 41, 26, 46, 43, 30, 10, 45, 38, 13, 36, 23, 40, 3, 34, 39, 8, 20, 18, 25, 22, 49, 14, 6, 4, 17, 15, 27, 11, 42, 31]
Array{Int64,1}[[5, 47, 2, 48, 29, 12, 19, 21, 35, 16, 28, 24, 32, 44, 7, 50, 33, 37, 1, 9, 41, 26, 46, 43, 30, 10, 45, 38, 13, 36], [23, 40, 3, 34, 39, 8, 20, 18, 25, 22], [49, 14, 6, 4, 17, 15, 27], [11, 42, 31]]
16 1 2.3377169175375796
16 2 1.337886850188108
16 3 -13.261910620

9 1 0.9095924725619726
9 2 0.6425560605433711
9 3 -12.28035401365599
9 4 -70.71499636027049
[13, 15, 48, 7, 2, 45, 32, 46, 9, 19, 33, 26, 14, 23, 1, 25, 28, 31, 20, 18, 11, 40, 21, 36, 49, 6, 39, 30, 44, 34, 43, 35, 8, 16, 50, 4, 3, 38, 22, 17, 47, 41, 42, 37, 12, 24, 27, 29, 10, 5]
Array{Int64,1}[[13, 15, 48, 7, 2, 45, 32, 46, 9, 19, 33, 26, 14, 23, 1, 25, 28, 31, 20, 18, 11, 40, 21, 36, 49, 6, 39, 30, 44, 34], [43, 35, 8, 16, 50, 4, 3, 38, 22, 17], [47, 41, 42, 37, 12, 24, 27], [29, 10, 5]]
10 1 1.284555677521606
10 2 -2.882345471251354
10 3 -10.323632806703378
10 4 -34.81839320752664
[32, 37, 20, 15, 19, 41, 43, 6, 27, 10, 17, 26, 39, 9, 50, 33, 16, 14, 23, 1, 25, 28, 31, 35, 7, 5, 34, 2, 18, 47, 8, 13, 42, 48, 46, 49, 40, 45, 21, 11, 36, 24, 12, 38, 30, 22, 4, 44, 3, 29]
Array{Int64,1}[[32, 37, 20, 15, 19, 41, 43, 6, 27, 10, 17, 26, 39, 9, 50, 33, 16, 14, 23, 1, 25, 28, 31, 35, 7, 5, 34, 2, 18, 47], [8, 13, 42, 48, 46, 49, 40, 45, 21, 11], [36, 24, 12, 38, 30, 22, 4], [44, 3, 29]]


1 1 0.9095924725619726
1 2 0.6425560605433711
1 3 -12.28035401365599
1 4 -70.71499636027049
2 1 -0.8359608032629973
2 2 0.7211195899606477
2 3 -34.85695393163233
2 4 -16.77312154268213
3 1 1.284555677521606
3 2 -2.882345471251354
3 3 -10.323632806703378
3 4 -34.81839320752664
4 1 2.1356905291624293
4 2 0.6727402796574706
4 3 -12.67254594026995
4 4 -34.81839320752664
5 1 2.2003152871223715
5 2 0.8187346640932653
5 3 -11.663622066624642
5 4 -34.81839320752664
fitness of crossover..
[7, 15, 24, 42, 47, 49, 32, 46, 9, 38, 33, 26, 1, 25, 28, 31, 20, 21, 27, 18, 2, 48, 14, 10, 50, 13, 36, 29, 22, 30, 45, 23, 19, 40, 39, 11, 41, 6, 37, 44, 8, 12, 43, 35, 34, 16, 17, 4, 3, 5]
Array{Int64,1}[[7, 15, 24, 42, 47, 49, 32, 46, 9, 38, 33, 26, 1, 25, 28, 31, 20, 21, 27, 18, 2, 48, 14, 10, 50, 13, 36, 29, 22, 30], [45, 23, 19, 40, 39, 11, 41, 6, 37, 44], [8, 12, 43, 35, 34, 16, 17], [4, 3, 5]]
6 1 0.9196403368592876
6 2 1.7680633687745502
6 3 -11.955686594889995
6 4 1753.9051139395701
[15, 48, 24, 42,

1 1 0.9095924725619726
1 2 0.6425560605433711
1 3 -12.28035401365599
1 4 -70.71499636027049
2 1 0.8677867874302314
2 2 1.108173845653214
2 3 -6.598143792712173
2 4 -70.71499636027049
3 1 2.0523436993088
3 2 -1.5919951658965124
3 3 -11.663622066624642
3 4 -54.26432321273989
4 1 -0.8359608032629973
4 2 0.7211195899606477
4 3 -34.85695393163233
4 4 -16.77312154268213
5 1 1.284555677521606
5 2 -2.882345471251354
5 3 -10.323632806703378
5 4 -34.81839320752664
fitness of crossover..
[7, 15, 48, 24, 28, 31, 6, 10, 17, 49, 16, 39, 46, 9, 19, 33, 26, 14, 23, 1, 25, 4, 43, 42, 47, 22, 2, 32, 20, 18, 34, 27, 30, 45, 13, 36, 40, 11, 41, 37, 44, 8, 12, 35, 21, 50, 38, 3, 29, 5]
Array{Int64,1}[[7, 15, 48, 24, 28, 31, 6, 10, 17, 49, 16, 39, 46, 9, 19, 33, 26, 14, 23, 1, 25, 4, 43, 42, 47, 22, 2, 32, 20, 18], [34, 27, 30, 45, 13, 36, 40, 11, 41, 37], [44, 8, 12, 35, 21, 50, 38], [3, 29, 5]]
6 1 1.187451803829459
6 2 -0.9169377237412091
6 3 -10.35399635838941
6 4 -70.71499636027049
[13, 22, 15, 41, 24,

1 1 1.0393948540092421
1 2 -1.5919951658965124
1 3 -11.663622066624642
1 4 -70.71499636027049
2 1 0.9095924725619726
2 2 0.6425560605433711
2 3 -12.28035401365599
2 4 -70.71499636027049
3 1 1.187451803829459
3 2 -0.9169377237412091
3 3 -10.35399635838941
3 4 -70.71499636027049
4 1 0.8677867874302314
4 2 1.108173845653214
4 3 -6.598143792712173
4 4 -70.71499636027049
5 1 0.978345251598109
5 2 -0.9346858745746567
5 3 -11.518447698799585
5 4 -54.26432321273989
fitness of crossover..
[7, 15, 48, 24, 33, 31, 6, 10, 17, 49, 16, 46, 9, 19, 28, 26, 14, 8, 1, 25, 4, 43, 42, 47, 22, 2, 32, 5, 18, 34, 27, 45, 13, 30, 37, 39, 44, 23, 12, 36, 40, 11, 41, 35, 21, 50, 38, 3, 29, 20]
Array{Int64,1}[[7, 15, 48, 24, 33, 31, 6, 10, 17, 49, 16, 46, 9, 19, 28, 26, 14, 8, 1, 25, 4, 43, 42, 47, 22, 2, 32, 5, 18, 34], [27, 45, 13, 30, 37, 39, 44, 23, 12, 36], [40, 11, 41, 35, 21, 50, 38], [3, 29, 20]]
6 1 1.2315162412590537
6 2 0.5728681465702641
6 3 -13.196722967986533
6 4 -13.83084387977276
[15, 48, 7, 24, 

1 1 0.9942588578332319
1 2 -2.7172120750416573
1 3 -19.858240453064237
1 4 -70.71499636027049
2 1 1.0359255057109036
2 2 -2.439631379634351
2 3 -11.287325092101707
2 4 -70.71499636027049
3 1 1.0393948540092421
3 2 -1.5919951658965124
3 3 -11.663622066624642
3 4 -70.71499636027049
4 1 0.9095924725619726
4 2 0.6425560605433711
4 3 -12.28035401365599
4 4 -70.71499636027049
5 1 1.187451803829459
5 2 -0.9169377237412091
5 3 -10.35399635838941
5 4 -70.71499636027049
fitness of crossover..
[7, 15, 48, 24, 42, 47, 22, 2, 32, 46, 9, 19, 33, 40, 14, 1, 25, 28, 31, 20, 23, 21, 27, 49, 30, 10, 45, 38, 13, 36, 18, 16, 26, 41, 6, 37, 34, 44, 8, 17, 11, 4, 43, 39, 12, 35, 50, 3, 29, 5]
Array{Int64,1}[[7, 15, 48, 24, 42, 47, 22, 2, 32, 46, 9, 19, 33, 40, 14, 1, 25, 28, 31, 20, 23, 21, 27, 49, 30, 10, 45, 38, 13, 36], [18, 16, 26, 41, 6, 37, 34, 44, 8, 17], [11, 4, 43, 39, 12, 35, 50], [3, 29, 5]]
6 1 0.9733235674999225
6 2 0.19171830083968988
6 3 -5.283129383909538
6 4 -70.71499636027049
[7, 15, 24, 4